<a href="https://colab.research.google.com/github/Victor074/Examen_Aprendizaje_Profundo_VICTOR/blob/main/Examen_victor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Victor Alberto Lopez Cardona

Exp. 747175

In [ ]:
!unzip /content/drive/MyDrive/ML/m3ex02-data.zip

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import os
import numpy as np
import torch.optim as optim

En el data loader hago un resize para las imagenes de 256*256, y defino los labels (COVID-19, Non-COVID, Noraml) para que se puedan agregar a la data

In [7]:
#UNET for processing lung masks
#RESNET for classifying radiography

import os
from PIL import Image
import torch
from torchvision import transforms

def get_data_loader(batch_size=16):
    label_mapping = {
        'COVID-19': 0,
        'Non-COVID': 1,
        'Normal': 2
    }
    transform = {
        'image': transforms.Compose([
            transforms.Resize([256,256]),
            transforms.ToTensor(),
        ]),
        'mask': transforms.Compose([
            transforms.Resize([256,256],interpolation=transforms.InterpolationMode.NEAREST),
            transforms.ToTensor(),
        ])
    }
    path = ""
    train_paths = [(path+"/content/m3ex02-data/Train/COVID-19/images", path + "/content/m3ex02-data/Train/COVID-19/lung masks"),
                   (path+"/content/m3ex02-data/Train/Non-COVID/images",path+"/content/m3ex02-data/Train/Non-COVID/lung masks"),
                   (path+"/content/m3ex02-data/Train/Normal/images",path+"/content/m3ex02-data/Train/Normal/lung masks")]
    test_paths = [(path+"/content/m3ex02-data/Test/COVID-19/images", path + "/content/m3ex02-data/Test/COVID-19/lung masks"),
                  (path+"/content/m3ex02-data/Test/Non-COVID/images",path+"/content/m3ex02-data/Test/Non-COVID/lung masks"),
                  (path+"/content/m3ex02-data/Test/Normal/images",path+"/content/m3ex02-data/Test/Normal/lung masks")]
    val_paths = [(path+"/content/m3ex02-data/Val/COVID-19/images", path + "/content/m3ex02-data/Val/COVID-19/lung masks"),
                 (path+"/content/m3ex02-data/Val/Non-COVID/images",path+"/content/m3ex02-data/Val/Non-COVID/lung masks"),
                 (path+"/content/m3ex02-data/Val/Normal/images",path+"/content/m3ex02-data/Val/Normal/lung masks")]
    all_paths = [train_paths,test_paths,val_paths]
    dataloader = []
    for one_path in all_paths:
        data = []
        for radio,mask in one_path:
            radiotemp = os.path.split(radio)
            label = label_mapping[os.path.basename(radiotemp[0])]
            fsortlist = [file for file in sorted(os.listdir(radio))]
            for imgfile in fsortlist:
                image = transform["image"](Image.open(os.path.join(radio,imgfile)))
                imagemask = transform["mask"](Image.open(os.path.join(mask,imgfile)))
                data.append((image, imagemask, label))
        dataloader.append(torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=True, num_workers=4))
    return dataloader




Utilizo un batch size de 4 y num_workers de 4 para agilizar el proceso de entrenamiento

In [8]:
all_data = get_data_loader(batch_size=20)
#/content/m3ex02-data/Train/COVID-19/images

En el dataloader estoy procesando los 3 paths existentes para train, validation, testing, por lo que regreso una lista conteniendo toda la informacion

In [9]:
trainloader = all_data[0]
testloader = all_data[1]
valloader = all_data[2]

Para este modelo estoy usando una resnet18 para que sea mas sencillo de entrenar y modifico la primera capa para que pueda recibir un canalya que la imagen de entrada esta en escala de grises, y despues conecto una UNeT pre entrenada donde tambien cambie la entrada de la primera capa para que pudiera ser compatible, y al final agregue un dropout para prevenir overfitting.

In [10]:
import torch
import torch.nn as nn


import torch
import torch.nn as nn
import torchvision.models as models

class myunet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(myunet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.resnet_features = nn.Sequential(*list(self.resnet.children())[:-2])
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 3)

        self.pretrained_unet = models.segmentation.deeplabv3_resnet50(pretrained=True)
        self.pretrained_unet.backbone.conv1 = nn.Conv2d(512, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # for param in self.pretrained_unet.backbone.conv1.parameters():
        #   param.requires_grad = False
        self.conv = nn.Conv2d(64, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 3)

        self.dropout1 = nn.Dropout(p=0.5)


    def forward(self, x):
        x = self.resnet_features(x)

        #x = self.pretrained_unet(x)

        upconvfeatures = self.pretrained_unet.backbone.conv1(x)
        mask  = nn.Sigmoid()(self.conv(upconvfeatures))
        mask_rescaled = nn.functional.interpolate(mask, size=(256, 256), mode='bilinear', align_corners=False) # mask a size de 256x 256

        pool = self.avgpool(upconvfeatures)
        flatten = torch.flatten(pool, 1)
        flatten = self.dropout1(flatten)
        label = nn.Softmax(dim=1)(self.fc(flatten))

        return mask_rescaled, label





Para agilizar aun mas el proceso de entrenamiento y validacion estoy mandando mi modelo y variables a cuda

In [12]:
import torch.optim as optim
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"DEVICE: {device}")
model = myunet()
model = model.to(device)
epochs = 10
criterion_1 = nn.BCELoss()
criterion_2 = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = []
    for images, masks, labels in trainloader:
        images, masks, labels = images.to(device), masks.to(device), labels.to(device)
        optimizer.zero_grad()
        mask_out, label_out = model(images)
        loss = criterion_1(mask_out, masks) + criterion_2(label_out, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted_labels = torch.max(label_out, 1)
        accuracy = (predicted_labels == labels).float().mean()
        train_accuracy.append(accuracy)

    val_loss_1 = 0.0
    val_loss_2 = 0.0
    val_correct = 0
    total_val = 0
    model.eval()
    with torch.no_grad():
        for val_images, val_mask, val_labels in valloader:
            val_images, val_mask, val_labels = val_images.to(device), val_mask.to(device), val_labels.to(device)
            val_mask_out, val_label_out = model(val_images)
            val_loss_1 += criterion_1(val_mask_out, val_mask).item()
            val_loss_2 += criterion_2(val_label_out, val_labels).item()
            _, predicted_labels = torch.max(val_label_out, 1)
            val_correct += (predicted_labels == val_labels).sum().item()
            total_val += val_labels.size(0)

    val_loss = (val_loss_1 + val_loss_2) / (len(valloader) * 2)
    val_accuracy = val_correct / total_val

    print(f'Epoch {epoch+1}, Train Accuracy: {sum(train_accuracy) / len(train_accuracy)}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}')

# Test
model.eval()
test_accuracy = []
with torch.no_grad():
    for test_images, test_mask, test_labels in testloader:
        test_images, test_mask, test_labels = test_images.to(device), test_mask.to(device), test_labels.to(device)
        _, test_label_out = model(test_images)
        _, predicted_labels = torch.max(test_label_out, 1)
        test_accuracy.append((predicted_labels == test_labels).float().mean())

print(f'Test Accuracy: {sum(test_accuracy) / len(test_accuracy)}')





DEVICE: cuda
Epoch 1, Train Accuracy: 0.5315837264060974, Val Loss: 0.679998889510482, Val Accuracy: 0.6075318441942035
Epoch 2, Train Accuracy: 0.4541586637496948, Val Loss: 1.2682648898820597, Val Accuracy: 0.37013106885730107
Epoch 3, Train Accuracy: 0.4376460611820221, Val Loss: 0.7610635900959317, Val Accuracy: 0.5172604762783829
Epoch 4, Train Accuracy: 0.5262892842292786, Val Loss: 0.6912172655557794, Val Accuracy: 0.6016245154144361
Epoch 5, Train Accuracy: 0.6122164130210876, Val Loss: 0.6295172279061426, Val Accuracy: 0.6773121654052058
Epoch 6, Train Accuracy: 0.6531312465667725, Val Loss: 0.6306127746386722, Val Accuracy: 0.6697434004061289
Epoch 7, Train Accuracy: 0.6619862914085388, Val Loss: 0.6306657191242239, Val Accuracy: 0.6845117223555474
Epoch 8, Train Accuracy: 0.6947054862976074, Val Loss: 0.5842614591781503, Val Accuracy: 0.757614916005169
Epoch 9, Train Accuracy: 0.7079492211341858, Val Loss: 0.754668644048631, Val Accuracy: 0.5578733616392837
Epoch 10, Train A

La siguiente celda es usada para guardar el modelo

In [13]:
model_save_path = 'model_unet_and_resnet.pth'
torch.save(model.state_dict(), model_save_path)
